# 飞书机器人上手教程 (1) — 快速开始

本 Notebook 演示如何初始化 `LarkBot`，并向指定用户发送文本、富文本、图片和卡片消息。

## 前置条件

1. 在 [飞书开放平台](https://open.feishu.cn/) 创建一个**企业自建应用**，记录 App ID 和 App Secret。
2. 在应用「权限管理」中申请 `im:message` 权限（发送消息）。
3. 将凭证写入环境变量或项目根目录的 `.env` 文件：

```
FEISHU_APP_ID=cli_xxxxxxxx
FEISHU_APP_SECRET=xxxxxxxxxxxxxxxx
```

In [ ]:
# 安装依赖（首次运行）
# !pip install chattool lark-oapi python-dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # 从 .env 文件加载环境变量

from chattool.tools.lark import LarkBot

# LarkBot 自动从环境变量读取 FEISHU_APP_ID / FEISHU_APP_SECRET
bot = LarkBot()
print("LarkBot 初始化成功")

## 1. 验证机器人身份

In [ ]:
import json

resp = bot.get_bot_info()
if resp.code == 0:
    info = json.loads(resp.raw.content).get("bot", {})
    print(f"机器人名称: {info.get('app_name')}")
    print(f"激活状态:   {info.get('activate_status')} (2=已激活)")
    print(f"机器人 open_id: {info.get('open_id')}")
else:
    print(f"获取失败: {resp.code} {resp.msg}")

## 2. 发送文本消息

`receive_id_type` 可选值：`open_id` / `user_id` / `union_id` / `email` / `chat_id`

In [ ]:
TEST_USER = "rexwzh"       # user_id
ID_TYPE   = "user_id"

resp = bot.send_text(TEST_USER, ID_TYPE, "👋 你好，这是来自 chattool 的第一条消息！")

if resp.success():
    print(f"✅ 发送成功，message_id = {resp.data.message_id}")
else:
    print(f"❌ 发送失败: {resp.code} {resp.msg}")

### 文本中 @ 用户

In [ ]:
# @ 指定用户（user_id 对应的 at 标签）
resp = bot.send_text(TEST_USER, ID_TYPE, "<at user_id=\"rexwzh\">Rex</at> 有个任务需要处理")
print("发送状态:", "✅" if resp.success() else f"❌ {resp.msg}")

## 3. 发送富文本消息 (Post)

富文本支持多语言、标题、链接、@用户、内嵌图片等。

In [ ]:
post_content = {
    "zh_cn": {
        "title": "📋 项目进度更新",
        "content": [
            [
                {"tag": "text", "text": "本周进度："},
                {"tag": "a", "text": "查看详情", "href": "https://open.feishu.cn"},
            ],
            [
                {"tag": "text", "text": "✅ 已完成：需求文档"},
            ],
            [
                {"tag": "text", "text": "🔄 进行中：前端开发"},
            ],
        ],
    }
}

resp = bot.send_post(TEST_USER, ID_TYPE, post_content)
print("发送状态:", "✅" if resp.success() else f"❌ {resp.msg}")

## 4. 发送交互卡片 (Interactive)

卡片是飞书最强大的消息形式，支持按钮、下拉框等控件。

In [ ]:
import time

card = {
    "config": {"wide_screen_mode": True},
    "header": {
        "title": {"tag": "plain_text", "content": "🔔 任务提醒"},
        "template": "blue",
    },
    "elements": [
        {
            "tag": "div",
            "text": {
                "tag": "lark_md",
                "content": f"**发起人**: Rex\n**任务**: 完成飞书机器人文档\n**截止**: 今日 18:00\n\n当前时间：`{time.strftime('%H:%M:%S')}`",
            },
        },
        {"tag": "hr"},
        {
            "tag": "action",
            "actions": [
                {
                    "tag": "button",
                    "text": {"tag": "plain_text", "content": "✅ 确认"},
                    "type": "primary",
                    "value": {"action": "confirm", "task_id": "task_001"},
                },
                {
                    "tag": "button",
                    "text": {"tag": "plain_text", "content": "⏰ 稍后处理"},
                    "type": "default",
                    "value": {"action": "snooze", "task_id": "task_001"},
                },
            ],
        },
    ],
}

resp = bot.send_card(TEST_USER, ID_TYPE, card)
if resp.success():
    card_msg_id = resp.data.message_id
    print(f"✅ 卡片发送成功，message_id = {card_msg_id}")
else:
    print(f"❌ 发送失败: {resp.msg}")

## 5. 引用回复已有消息

In [ ]:
# 先发一条消息
orig = bot.send_text(TEST_USER, ID_TYPE, "[原始消息] 请在下方回复")
orig_id = orig.data.message_id

# 引用回复
resp = bot.reply(orig_id, "✅ 这是对原始消息的引用回复")
print("发送状态:", "✅" if resp.success() else f"❌ {resp.msg}")

## 6. 获取群组信息

In [ ]:
# 如果有群 chat_id，可以查询群信息
# chat_id = "oc_xxxxx"
# resp = bot.get_chat_info(chat_id)
# if resp.success():
#     print(f"群名称: {resp.data.name}")
#     print(f"成员数: {resp.data.member_count}")

print("（需要有群的 chat_id 才能查询，请替换上方变量后取消注释运行）")

## 小结

| 方法 | 说明 |
|------|------|
| `bot.get_bot_info()` | 验证凭证，获取机器人基本信息 |
| `bot.send_text(id, type, text)` | 发送纯文本消息 |
| `bot.send_post(id, type, content)` | 发送富文本消息 |
| `bot.send_card(id, type, card)` | 发送交互卡片 |
| `bot.reply(message_id, text)` | 引用回复某条消息 |
| `bot.reply_card(message_id, card)` | 引用回复（卡片） |

下一篇：[02 — 接收消息与路由](lark_02_receiving.ipynb)